In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


22/11/06 09:56:13 WARN Utils: Your hostname, DESKTOP-TVDS9T6 resolves to a loopback address: 127.0.1.1; using 172.29.38.221 instead (on interface eth0)
22/11/06 09:56:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/06 09:56:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd
import dask.dataframe as dd

X = pd.DataFrame(zip(range(100), range(100)), columns=['x','y'])
X_dask = dd.from_pandas(X, npartitions=1)
X_spark = spark.createDataFrame(X)

/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [8]:
from dfender import distributable

class Test:
    @distributable(pd.Series)
    def predict(self, X: pd.Series) -> pd.Series:
        return X - 2
        # return (pd.to_datetime('2000-01-01').to_period('M') + X).dt.to_timestamp()

    @distributable((pd.Series, pd.DataFrame))
    def rowmax(self, X: pd.DataFrame):
        return X.max(axis=1)


est = Test()
# est.predict(X['x'])
# est.predict(X_dask['x']).compute()
# est.predict(X_spark.select(['x']))
# est.rowmax(X)
# est.rowmax(X_)
# Test.test.funcs
est.predict.funcs

{(pandas.core.series.Series,): <function __main__.Test.predict(self, X: pandas.core.series.Series) -> pandas.core.series.Series>,
 (dask.dataframe.core.Series,): <function dfender.materializers.core.MaterializerInterface.map.<locals>.wrapper(_, df: 'Any', *args, **kwargs)>,
 (pyspark.sql.dataframe.DataFrame,): <function dfender.materializers.core.MaterializerInterface.map.<locals>.wrapper(_, df: 'Any', *args, **kwargs)>}

In [9]:
est.predict(X['x'])

0     -2
1     -1
2      0
3      1
4      2
      ..
95    93
96    94
97    95
98    96
99    97
Name: x, Length: 100, dtype: int64

In [10]:
est.predict(X_dask['x']).compute()

0     -2
1     -1
2      0
3      1
4      2
      ..
95    93
96    94
97    95
98    96
99    97
Name: x, Length: 100, dtype: int64

In [12]:
est.predict(X_spark.select(['x'])).collect()

/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/ed/projects/dfender/.venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


x bigint


22/11/06 09:57:59 ERROR Executor: Exception in task 0.0 in stage 2.0 (TID 48)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_22713/557874211.py", line 6, in predict
TypeError: unsupported operand type(s) for -: 'map' and 'int'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:101)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:50)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:36

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_22713/557874211.py", line 6, in predict
TypeError: unsupported operand type(s) for -: 'map' and 'int'


22/11/06 09:57:59 WARN TaskSetManager: Lost task 3.0 in stage 2.0 (TID 51) (172.29.38.221 executor driver): TaskKilled (Stage cancelled)
22/11/06 09:57:59 WARN TaskSetManager: Lost task 5.0 in stage 2.0 (TID 53) (172.29.38.221 executor driver): TaskKilled (Stage cancelled)
